# **Checkpoint 2**

In [2]:
import pandas as pd

file_path = '/content/drive/MyDrive/AVD/bestsellers_with_categories_2022_03_27.csv'
df = pd.read_csv(file_path)

# Data Understanding

Sumber data: [Amazon Top 50 Bestselling Books 2009 - 2022](https://www.kaggle.com/datasets/chriskachmar/amazon-top-50-bestselling-books-2009-2022)

## Pemeriksaan Data

### Tabel Data

In [3]:
df

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,"Act Like a Lady, Think Like a Man: What Men Re...",Steve Harvey,4.6,5013,17,2009,Non Fiction
1,Arguing with Idiots: How to Stop Small Minds a...,Glenn Beck,4.6,798,5,2009,Non Fiction
2,"Breaking Dawn (The Twilight Saga, Book 4)",Stephenie Meyer,4.6,9769,13,2009,Fiction
3,Crazy Love: Overwhelmed by a Relentless God,Francis Chan,4.7,1542,14,2009,Non Fiction
4,Dead And Gone: A Sookie Stackhouse Novel (Sook...,Charlaine Harris,4.6,1541,4,2009,Fiction
...,...,...,...,...,...,...,...
695,The Wonderful Things You Will Be,Emily Winfield Martin,4.9,20920,9,2022,Fiction
696,Ugly Love: A Novel,Colleen Hoover,4.7,33929,10,2022,Fiction
697,Verity,Colleen Hoover,4.6,71826,11,2022,Fiction
698,What to Expect When You're Expecting,Heidi Murkoff,4.8,27052,13,2022,Non Fiction


### Struktur Data

Ringkasan singkat dataset

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         700 non-null    object 
 1   Author       700 non-null    object 
 2   User Rating  700 non-null    float64
 3   Reviews      700 non-null    int64  
 4   Price        700 non-null    int64  
 5   Year         700 non-null    int64  
 6   Genre        700 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 38.4+ KB


Jumlah baris dan kolom pada dataset (Baris, Kolom)

In [6]:
df.shape

(700, 7)

Tipe data pada setiap kolom

In [9]:
df.dtypes

,0
Name,object
Author,object
User Rating,float64
Reviews,int64
Price,int64
Year,int64
Genre,object


Statistik deskriptif pada dataset

In [10]:
df.describe()

,User Rating,Reviews,Price,Year
count,700.000000,700.000000,700.000000,700.000000
mean,4.639857,19255.195714,12.700000,2015.500000
std,0.218586,23613.443875,9.915162,4.034011
min,3.300000,37.000000,0.000000,2009.000000
25%,4.500000,4987.250000,7.000000,2012.000000
50%,4.700000,10284.000000,11.000000,2015.500000
75%,4.800000,23358.000000,15.000000,2019.000000
max,4.900000,208917.000000,105.000000,2022.000000


Analisis korelasi antar kolom

In [13]:
df.corr(numeric_only=True)

,User Rating,Reviews,Price,Year
User Rating,1.000000,0.080423,-0.157253,0.286601
Reviews,0.080423,1.000000,-0.114554,0.494279
Price,-0.157253,-0.114554,1.000000,-0.155266
Year,0.286601,0.494279,-0.155266,1.000000


# Data Preparation

Sumber data: [Amazon Top 50 Bestselling Books 2009 - 2022](https://www.kaggle.com/datasets/chriskachmar/amazon-top-50-bestselling-books-2009-2022)

## Pengecekan Data

Pengecekan terhadap data untuk menemukan nilai yang hilang (Missing Values), duplikat, outliers, dan nilai yang tidak konsisten (Inconsistent Values)

### Missing Values

In [14]:
print((df.isna().sum() / len(df)) * 100)

Name           0.0
Author         0.0
User Rating    0.0
Reviews        0.0
Price          0.0
Year           0.0
Genre          0.0
dtype: float64


Data tidak ada yang kosong dan tidak diperlukan untuk pemgimputasian data

### Data Duplikat

In [15]:
df[df.duplicated()]

,Name,Author,User Rating,Reviews,Price,Year,Genre


Data tidak ada yang duplikat dan tidak diperlukan untuk penghapusan data yang duplikat

### Outliers

Melakukan pengecekan outliers

In [16]:
results = []

cols = df.select_dtypes(include=['float64', 'int64'])

for col in cols:
  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - 1.5*iqr
  upper_bound = q3 + 1.5*iqr
  outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
  percent_outliers = (len(outliers)/len(df))*100
  results.append({'Kolom': col, 'Persentase Outliers': percent_outliers})

results_df = pd.DataFrame(results)
results_df.set_index('Kolom', inplace=True)
results_df = results_df.rename_axis(None, axis=0).rename_axis('Kolom', axis=1)

display(results_df)

Kolom,Persentase Outliers
User Rating,3.142857
Reviews,9.571429
Price,5.428571
Year,0.000000


### Inconsistent Value

Setelah dianalisis dapat diketahui bahwa tidak ada data yang tidak konsisten pada dataset ini

## Penanganan Data

Melakukan penangan pada outlier data

In [19]:
columns_to_impute = ["User Rating", "Reviews", "Price"]

for col in columns_to_impute:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df.loc[:, col] = df[col].clip(lower=lower_bound, upper=upper_bound)

Melakukan pengecekan ulang

In [20]:
results = []

cols = df.select_dtypes(include=['float64', 'int64'])

for col in cols:
  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - 1.5*iqr
  upper_bound = q3 + 1.5*iqr
  outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
  percent_outliers = (len(outliers)/len(df))*100
  results.append({'Kolom': col, 'Persentase Outliers': percent_outliers})

results_df = pd.DataFrame(results)
results_df.set_index('Kolom', inplace=True)
results_df = results_df.rename_axis(None, axis=0).rename_axis('Kolom', axis=1)

display(results_df)

Kolom,Persentase Outliers
User Rating,0.0
Reviews,0.0
Price,0.0
Year,0.0


## Construct Data

Menambahkan kolom kategori harga dari murah, menengah, hingga mahal

In [24]:
df['Price Category'] = pd.cut(
    df['Price'],
    bins=[1, 10, 20, df['Price'].max()],
    labels=['Murah', 'Menengah', 'Mahal']
)

df.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre,Price Category
0,"Act Like a Lady, Think Like a Man: What Men Re...",Steve Harvey,4.6,5013.0,17,2009,Non Fiction,Menengah
1,Arguing with Idiots: How to Stop Small Minds a...,Glenn Beck,4.6,798.0,5,2009,Non Fiction,Murah
2,"Breaking Dawn (The Twilight Saga, Book 4)",Stephenie Meyer,4.6,9769.0,13,2009,Fiction,Menengah
3,Crazy Love: Overwhelmed by a Relentless God,Francis Chan,4.7,1542.0,14,2009,Non Fiction,Menengah
4,Dead And Gone: A Sookie Stackhouse Novel (Sook...,Charlaine Harris,4.6,1541.0,4,2009,Fiction,Murah


## Data Reduction

Menghapus kolom Year atau tahun karena data tersebut tidak terlalu dibutuhkan untuk menganalisis harga dan popularitas pada buku

In [ ]:
df = df.drop('Year', axis=1)

Maaf terjadi error karena tidak sengaja run kode nya 2 kali

Berikut hasil dari penghapusan kolom 'Year'

In [29]:
df

,Name,Author,User Rating,Reviews,Price,Genre,Price Category
0,"Act Like a Lady, Think Like a Man: What Men Re...",Steve Harvey,4.6,5013.000,17,Non Fiction,Menengah
1,Arguing with Idiots: How to Stop Small Minds a...,Glenn Beck,4.6,798.000,5,Non Fiction,Murah
2,"Breaking Dawn (The Twilight Saga, Book 4)",Stephenie Meyer,4.6,9769.000,13,Fiction,Menengah
3,Crazy Love: Overwhelmed by a Relentless God,Francis Chan,4.7,1542.000,14,Non Fiction,Menengah
4,Dead And Gone: A Sookie Stackhouse Novel (Sook...,Charlaine Harris,4.6,1541.000,4,Fiction,Murah
...,...,...,...,...,...,...,...
695,The Wonderful Things You Will Be,Emily Winfield Martin,4.9,20920.000,9,Fiction,Murah
696,Ugly Love: A Novel,Colleen Hoover,4.7,33929.000,10,Fiction,Murah
697,Verity,Colleen Hoover,4.6,50914.125,11,Fiction,Menengah
698,What to Expect When You're Expecting,Heidi Murkoff,4.8,27052.000,13,Non Fiction,Menengah
